## Компьютерное зрение в робототехнике | Домашнее задание 2

В этом домашнем задании нужно будет реализовать подсчет числа пальцев, которые показывает человек на видео fingers.mov из папки data.

Для удобства реализации значительная часть кода уже написана. Внимательно изнакомьтесь с заготовками, поймите, что происходит в коде и для чего нужна каждая конкретная функция.

Есть много вариантов, как можно подступиться к этой задаче. Приведем некоторые из них:

**Скелетонизация**
- получение маски
- обработка (фильтрация шумов, сглаживание)
- удаление всего, кроме самой большой связной компоненты
- скелетонизация (можно загуглить opencv skeletonization)
- нахождение кончиков пальцев (filter2d)
- фильтрация кончиков пальцев

**Дефекты выпуклости**
- получение маски
- обработка (фильтрация шумов, сглаживание)
- удаление всего, кроме самой большой связной компоненты
- нахождение контура, построение грубой аппроксимации
- нахождение дефектов выпуклости (convexity defects)

**Морфология**
- получение маски
- обработка (фильтрация шумов, сглаживание)
- удаление всего, кроме самой большой связной компоненты
- морфологические операции top hat/black hat

Можно выбрать любой из этих способов, который по душе, или придумать свой. Нейронные сети и другие методы машинного обучения использовать в решении нельзя :)

***

<h2 style="color:#A7BD3F;">Секция 1: подготовка</h2>

Этот код представляет из себя в сущности обертку над функцией обработки кадра. Завершение выполнения производится с помощью нажатия кнопки 'q' (работает только для английского языка).

In [ ]:
import numpy as np
import cv2

class FrameProcessor:
    def __init__(self):
        pass
    
    def processing_loop(self, source, lth, hth, max_frame_num = -1,\
                        alternative_source="", save_to_file=""):
        i = 0
        results = []

        output_file = None
        
        #out = cv2.VideoWriter('outpy.avi', cv2.VideoWriter_fourcc('M','J','P','G'),
#                               30, (WINDX, WINDY))
#         out.write(canvas)
#         out.release()
        
        while (True):
            retval, frame = source.read()

            if (retval == False):
                print("Cannot read frame")
                
                if (alternative_source != ""):
                    print("Opening alternative source ", alternative_source)
                    source = cv2.VideoCapture(alternative_source)
                    continue
                
                else:
                    print("Exiting loop")
                    break

            result = self.process_frame(frame, lth, hth)
            
            results.append(result)

            key = cv2.waitKey(100) & 0xFF

            i += 1

            if (key == ord('q')):
                break
                        
            if (max_frame_num != -1 and i >= max_frame_num):
                break

        return results
    
    def process_frame(self, frame, lth, hth):
        return 5

***

<h2 style="color:#A7BD3F;">Секция 2: настройка цветового фильтра</h2>

Настройте параметры фильтрации по цвету. Это нужно делать в цветовом пространстве *HSV*. После этого запишите найденные параметры в *lth* и *hth*, их можно будет использовать позже.

In [ ]:
#############################################
# YOUR DEFAULT PARAMETERS BELOW
#############################################

lth, hth = (0, 47, 131), (25, 165, 255)
#(0, 85, 60) (255, 205, 233)

#############################################
# YOUR DEFAULT PARAMETERS ABOVE
#############################################

class ColorFilterTuning(FrameProcessor):
    def __init__(self):
        super().__init__()
        
        cv2.namedWindow("color_filter_parameters")
                
        cv2.createTrackbar('rl', 'color_filter_parameters', lth[0], 255, self.nothing)
        cv2.createTrackbar('gl', 'color_filter_parameters', lth[1], 255, self.nothing)
        cv2.createTrackbar('bl', 'color_filter_parameters', lth[2], 255, self.nothing)
        cv2.createTrackbar('rh', 'color_filter_parameters', hth[0], 255, self.nothing)
        cv2.createTrackbar('gh', 'color_filter_parameters', hth[1], 255, self.nothing)
        cv2.createTrackbar('bh', 'color_filter_parameters', hth[2], 255, self.nothing)

    def nothing(self, inp):
        pass
    
    def process_frame(self, frame, lth, hth):
        #frame = cv2.blur(frame, (20, 20))
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        low_th =  (cv2.getTrackbarPos('rl', 'color_filter_parameters'),
                   cv2.getTrackbarPos('gl', 'color_filter_parameters'),
                   cv2.getTrackbarPos('bl', 'color_filter_parameters'))
        
        high_th = (cv2.getTrackbarPos('rh', 'color_filter_parameters'),
                   cv2.getTrackbarPos('gh', 'color_filter_parameters'),
                   cv2.getTrackbarPos('bh', 'color_filter_parameters'))
        
        mask = cv2.inRange(hsv, low_th, high_th)
        
        connectivity = 4
        output = cv2.connectedComponentsWithStats(mask, connectivity, cv2.CV_32S)
        num_labels = output[0]
        labels = output[1]
        stats = output[2]
        a_max = stats[1, cv2.CC_STAT_AREA]
        for i in range(2, num_labels):
            if (a_max < stats[i, cv2.CC_STAT_AREA]):
                a_max = stats[i, cv2.CC_STAT_AREA]   
        for i in range(1, num_labels):
            #if (stats[i, cv2.CC_STAT_AREA] < th_area):
            a = stats[i, cv2.CC_STAT_AREA]
            w = stats[i, cv2.CC_STAT_WIDTH]
            h = stats[i, cv2.CC_STAT_HEIGHT]
            if ((a < a_max) or (a < 800)):
                mask[np.where(labels == i)] = 0

        ksz = 39 + 1
        kernel = np.ones((ksz, ksz), np.uint8) 
        tophat = cv2.morphologyEx(mask, cv2.MORPH_TOPHAT, kernel) #MORPH_BLACKHAT MORPH_TOPHAT
        ksz_op = 25 + 1
        kernel_op = np.ones((ksz_op, ksz_op), np.uint8) 
        morph = cv2.morphologyEx(tophat, cv2.MORPH_OPEN, kernel_op)
        cv2.imshow("MORPH", morph)
        
        #cv2.imshow("frame", tophat)
        cv2.imshow("frame", frame)
        cv2.imshow("mask", mask)
        
        return (low_th, high_th)

In [ ]:
import numpy as np
import cv2

video_file = "/Users/dududugin/Desktop/CVR23/data_videos_fingers.mov"

cam = cv2.VideoCapture(video_file)

#print(cam)
# frame_offset = 100
# cam.set(1, frame_offset)

tuner = ColorFilterTuning()

colors = tuner.processing_loop(cam, None, None, max_frame_num = -1,\
            alternative_source=video_file)
lth, hth = colors[-1]

print("Color filter parameters: ", lth, hth)
cam.release()
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(100)

Color filter parameters:  (0, 47, 131) (25, 165, 255)


113

***

<h2 style="color:#A7BD3F;">Секция 3: подсчет пальцев</h2>

Реализуйте алгоритм нахождения числа пальцев, закомментировав функцию, которая всегда возвращает 3 и написав свою. Вы можете использовать *lth* и *hth*, которые получили выше. Заготовленные функции для обработки масок можно использовать, а можно и не использовать.

In [ ]:


class FingersCounter(FrameProcessor):
    def __init__(self):
        super().__init__()

    def filter_cc(self, mask, area_th = -1):
        connectivity = 4
        output = cv2.connectedComponentsWithStats(mask, connectivity, cv2.CV_32S)
        num_labels = output[0]
        labels = output[1]
        stats = output[2]
        #centroids = output[3]

        if (num_labels < 1):
            return mask
        
        if (area_th == -1):
            max_area = 1
            max_label = 1
            
            for i in range(1, num_labels):
                area = stats[i, cv2.CC_STAT_AREA]
                
                if (area > max_area):
                    max_area = area
                    max_label = i
            
            for i in range(1, len(stats)):
                if (i != max_label):
                    mask[np.where(labels == i)] = 0
                    
        else:
            for i in range(len(stats)):
                area = stats[i, cv2.CC_STAT_AREA]

                if (area < area_th):
                    mask[np.where(labels == i)] = 0

        return mask
    
    def fill_holes (self, img):
        (h, w) = img.shape

        before_area = img.sum ()

        img_enlarged = np.zeros ((h + 2, w + 2), np.uint8)
        img_enlarged [1:h+1, 1:w+1] = img

        img_enl_not = cv2.bitwise_not (img_enlarged)
        th, im_th = cv2.threshold (img_enl_not, 220, 255, cv2.THRESH_BINARY_INV);

        im_floodfill = im_th.copy()

        h, w = im_th.shape[:2]
        mask = np.zeros((h+2, w+2), np.uint8)

        cv2.floodFill(im_floodfill, mask, (0,0), 255);
        im_floodfill_inv = cv2.bitwise_not(im_floodfill)
        im_out = im_th | im_floodfill_inv

        result = im_out [1:h-1, 1:w-1]

        #after_area = result.sum ()
        
        return result

    #def process_frame(self, frame, lth, hth):
       # cv2.imshow("frame", frame)
        
       # return 3
    def nothing(self, inp):
            pass
    
    def process_frame(self, frame, lth, hth):
        #############################################
        # YOUR CODE BELOW
        #############################################
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        mask = cv2.inRange(hsv, (0, 47, 131), (25, 165, 255))
        mask_stage = cv2.inRange(hsv, (0, 47, 131), (25, 165, 255))
        #фильтруем связанные компоненты, чтобы осталась только рука
        connectivity = 4
        output = cv2.connectedComponentsWithStats(mask, connectivity, cv2.CV_32S)
        num_labels = output[0]
        labels = output[1]
        stats = output[2]
        a_max = stats[1, cv2.CC_STAT_AREA]
        for i in range(2, num_labels):
            if (a_max < stats[i, cv2.CC_STAT_AREA]):
                a_max = stats[i, cv2.CC_STAT_AREA]   
        for i in range(1, num_labels):
            #if (stats[i, cv2.CC_STAT_AREA] < th_area):
            a = stats[i, cv2.CC_STAT_AREA]
            w = stats[i, cv2.CC_STAT_WIDTH]
            h = stats[i, cv2.CC_STAT_HEIGHT]
            if ((a < a_max) or (a < 800)):
                mask[np.where(labels == i)] = 0
        
        
        #применяем морфологию MORPH_TOPHAT, которая оставляет обрезки пальцев
        ksz = 37 + 1
        kernel = np.ones((ksz, ksz), np.uint8) 
        tophat = cv2.morphologyEx(mask, cv2.MORPH_TOPHAT, kernel) #MORPH_BLACKHAT MORPH_TOPHAT
        #применяем морфологию MORPH_OPEN, которая убирает почти все кроме пальцев
        ksz_op = 25 + 1
        kernel_op = np.ones((ksz_op, ksz_op), np.uint8) 
        morph = cv2.morphologyEx(tophat, cv2.MORPH_OPEN, kernel_op)
        
        
        connectivity = 5
        output_fin = cv2.connectedComponentsWithStats(morph, connectivity, cv2.CV_32S)
        fingers_num = output_fin[0] - 1
        
        stages_concat = np.concatenate((morph, tophat, mask_stage ), axis=1)
        cv2.imshow("frame", stages_concat)
        return fingers_num
        
        #############################################
        # YOUR CODE ABOVE
        #############################################
        
    #    cv2.imshow("stages", stages_concat)
        
    #    return fingers_num

In [ ]:
cam = cv2.VideoCapture(video_file)

finger_counter = FingersCounter()

fingers_num = finger_counter.processing_loop(cam, lth, hth)

print(fingers_num)

cam.release()
cv2.waitKey(0)
cv2.destroyAllWindows()
cv2.waitKey(100)

[5, 4, 0, 1, 1, 5, 4, 5, 0, 1, 1, 5, 5, 5, 5, 4, 3, 3, 3, 3, 2, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 2, 2, 2, 2, 1, 2, 3, 4, 3, 3, 3, 4, 5, 5, 5, 5, 5, 5, 5, 2, 2, 2, 2, 2, 1, 1, 1, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3, 3, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0]


-1

***

<h2 style="color:#A7BD3F;">Секция 4: оценивание</h2>

Решение, дающее значения метрики *accuracy* *0.5* и более, оцениваются полным баллом с линейной интерполяцией в области меньших значений.

Выполните эту клетку для получения своей оценки, изучите, как она была посчитана. Если в процессе выполнения задания появились вопросы или проблемы, пишите в чат или @elijahmipt в телеграме.

In [ ]:
reference_fingers_num = [5, 5, 1, 0, 0, 5, 5, 5, 0, 0, 0, 5, 5, 5, 5, 4, 3, 3,\
                         3, 3, 3, 3, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 2,\
                         2, 2, 2, 2, 3, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5,\
                         2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2,\
                         2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1,\
                         3, 4, 0, 0, 0, 1]

max_grade = 100

corr_num = 0

for r, s in zip(reference_fingers_num, fingers_num):
    if (r == s):
        corr_num += 1

acc = corr_num / len(reference_fingers_num)

#print("correct ", corr_num, " out of ", len(reference_fingers_num),
#      corr_num / len(reference_fingers_num))

grade = min(acc * 2, 1) * max_grade

print("Your grade is ", "\033[92m{}\033[0m".format(str(int(grade)) +\
        " out of " + str(max_grade) + "; " + str(corr_num) + " frames out of "
        + str(len(reference_fingers_num))))

Your grade is  100 out of 100; 62 frames out of 96


# GPT-3

In [ ]:
import cv2
import numpy as np

# Задаем видеофайл для обработки
cap = cv2.VideoCapture('/Users/dududugin/Desktop/CVR23/data_videos_fingers.mov')

while True:
    # Читаем кадр из видео
    ret, frame = cap.read()

    if ret:
        # Преобразуем кадр в HSV
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Определяем диапазон цветов для кожи
        lower_skin = np.array([0, 20, 70], dtype=np.uint8)
        upper_skin = np.array([20, 255, 255], dtype=np.uint8)

        # Создаем маску, которая выделяет область кожи на изображении
        mask = cv2.inRange(hsv, (0, 47, 131), (25, 165, 255))

        # Применяем морфологические операции для удаления шумов и заполнения дырок
        kernel = np.ones((5, 5), np.uint8)
        mask = cv2.erode(mask, kernel, iterations=1)
        mask = cv2.dilate(mask, kernel, iterations=1)
        cv2.imshow('mask', mask)

        # Находим компоненты связности на маске
        connectivity = 4
        output = cv2.connectedComponentsWithStats(mask, connectivity, cv2.CV_32S)

        # Получаем статистику по компонентам связности
        num_labels = output[0]
        stats = output[2]

        # Считаем количество пальцев
        finger_count = 0
        for i in range(1, num_labels):
            # Получаем размеры компоненты связности
            left = stats[i, cv2.CC_STAT_LEFT]
            top = stats[i, cv2.CC_STAT_TOP]
            width = stats[i, cv2.CC_STAT_WIDTH]
            height = stats[i, cv2.CC_STAT_HEIGHT]

            # Вычисляем соотношение ширины и высоты компоненты связности
            aspect_ratio = float(height) / width

            # Проверяем, что компонента связности представляет собой палец
            if 1.2 < aspect_ratio < 3.0 and width > 50:
                finger_count += 1

        # Выводим количество пальцев на изображении
        cv2.putText(frame, str(finger_count), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Отображаем обработанный кадр
        cv2.imshow('frame', frame)
        cv2.waitKey(1000)

        # Если нажата клавиша 'q', то выходим из цикла
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# Освобождаем ресурсы и закрываем окна
cap.release()
cv2.destroyAllWindows()
